# Glove-Char-Classifier

This notebook is part of my tutorial on creating custom GloVe embeddings. Please find the repository here: https://github.com/remo-help/character-embedding-with-glove

## Imports:
First we need to import sklearn and numpy.

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score

import numpy as np


## Reading in the embeddings

Unfortunately, sklearn by itself does not support GloVe vector files, so we need to write a little function that allows us to read in the embeddings we need.

This function will take a file with glove-style embeddings and return a dictionary where the keys are the word or character that is embedded and the value is the respective vector.

In [2]:
def glove(path):
        
    embeddings_dict={}
        
    f = open(path,'r',encoding='utf8') #reading in the input data
    vector_file = f.read()
    f.close()
    vector_file=vector_file.split("\n")
        
    for line in vector_file:
        if line[0]:
            line=line.split()
            token = line[0]
            vector=line[1:]
            vector= [float(x) for x in vector]
            vector = np.array(line[1:])
            vector = vector.astype('float64')
            embeddings_dict[token]=vector
            
    return embeddings_dict
        
        

## The Classifier

Next, we have to define the classifier that we intend to use. This classifier will have a variety of attributes and functions:

### model
We will use the sklearn LogisticRegression model.
### label_encoder
We will use the sklearn LabelEncoder
### vectorizer
For comparison, we will also use the sklearn CountVectorizer

### train()
This function will train our classifier on the GloVe vectors we provide.
### train_count()
This function will train our classifier on the embeddings provided by the CountVectorizer of the sklearn library. This is a count-based embedding technique.
### predict_labels()
This function will take in test data and make predictions based on the test data. The output of the function will be encoded labels (integers). We can inverse_transform those labels with the LabelEncoder if we want to see the strings.
### predict_labels_count()
This is the same as above, except for the CountVectorizer embeddings.

In [3]:
class Classifier:
    def __init__(self):
        """
        Initializes the classifier.
        """
        self.label_encoder = LabelEncoder()
        
           
        self.vectorizer = CountVectorizer(analyzer='char', ngram_range=(1, 10)) #we are using the "word" parameter because our characters are already seperated

        
        self.model = LogisticRegression(solver="lbfgs", multi_class="multinomial", max_iter=5000,verbose=1)
    
    def train(self, vectors, train_data_path):
        """
        trains on the GloVe embeddings
        """

        f = open(train_data_path,'r',encoding="utf8") #reading in the input data
        input_string = f.read()
        f.close()
        
        input_list=input_string.split("\n") #storing each datum as a string in a list
        input_list=input_list[0:-2] #getting rid of the last empty newline
        feature_strings= []
        label_strings= []
        
        for datum in input_list:
            temp = datum.split("\t") #separating the word from its label
            feature_strings.append(temp[0])
            label_strings.append(temp[1])
        del input_list #deleting the initial list to be economic
        
        vector_list=[] #here we will store our feature vectors
        
        for feature in feature_strings: #selecting a feature
            feature = feature[0:-1].split(" ") #getting rid of trailing space and splitting on space
            temp_list = [] #here we will compile the feature vector
            for char in feature: #iterating over characters of the word
                if char in vectors.keys(): #making sure we are not running into unknown chars
                    vector = vectors[char] #getting the vector associated with the character
                    temp_list.append(vector)
            
            if len(temp_list)==0:
                print("cannot find:",feature)
                
            base = temp_list[0] #selecting the first character-vector

            for i in range(1,len(temp_list)): 
                base=np.add(base,temp_list[i]) #adding all other character vectors item-wise
            array=base/len(temp_list) #dividing by the total amount of characters
        
            vector_list.append(array) #putting the new and averaged array into our list
                
                
                
        
        x_train = vector_list #transforming the feature strings into glove vectors based on our pretrained embedding

        y_train = self.label_encoder.fit_transform(label_strings) #fitting and transforming the labels to integers
        
        if len(x_train)!=len(y_train): #making sure we have as many feature vectors as we have labels
            print("features:",len(x_train),"\n","labels:",len(y_train))
        
        self.model.fit(x_train, y_train)
        
        print("The classifier has finished training")
        
    def train_count(self, train_data_path):
        """
        This trains the classifier on vectors created by the CountVectorizer, as opposed to pretrained embeddings
        """

        f = open(train_data_path,'r',encoding="utf8") #reading in the input data
        input_string = f.read()
        f.close()
        
        input_list=input_string.split("\n") #storing each datum as a string in a list
        input_list=input_list[0:-2] #getting rid of the last empty newline
        feature_strings= []
        label_strings= []
        
        for datum in input_list:
            temp = datum.split("\t") #separating the word from its label
            feature_strings.append(temp[0])
            label_strings.append(temp[1])
        del input_list #deleting the initial list
        

        
        x_train = self.vectorizer.fit_transform(feature_strings) #transforming the feature strings into vectors with the count vectorizer

        y_train = self.label_encoder.fit_transform(label_strings) #fitting and transforming the labels to integers
        
        self.model.fit(x_train, y_train) #training the model
        
        print("The classifier has finished training (CountVectorizer)")
        
        
    
    def predict_labels(self, vectors, test_data_path):
        """
        Takes a testfile where tokens and labels are seperated with \t
        returns the sequence of gold-lables and the sequence of predicted labels in INTEGER FORM
        """
        f = open(test_data_path,'r',encoding="utf8") #reading in the input data
        input_string = f.read()
        f.close()
        
        input_list=input_string.split("\n") #storing each datum as a string in a list
        input_list=input_list[0:-2] #getting rid of the last empty newline
        feature_strings= []
        label_strings= []
        
        for datum in input_list:
            temp = datum.split("\t") #separating the word from its label
            feature_strings.append(temp[0])
            label_strings.append(temp[1])
        del input_list #deleting the initial list to be economic
        
        vector_list=[] #here we will store our feature vectors
        
        for feature in feature_strings: #selecting a feature
            feature = feature[0:-1].split(" ") #getting rid of trailing space and splitting on space
            temp_list = [] #here we will compile the feature vector
            for char in feature: #iterating over characters of the word
                if char in vectors.keys(): #making sure we are not running into unknown chars
                    vector = vectors[char] #getting the vector associated with the character
                    temp_list.append(vector)
            
            if len(temp_list)==0:
                print("cannot find:",feature)
                
            base = temp_list[0] #selecting the first character-vector

            for i in range(1,len(temp_list)): 
                base=np.add(base,temp_list[i]) #adding all other character vectors item-wise
            array=base/len(temp_list) #dividing by the total amount of characters
        
            vector_list.append(array) #putting the new and averaged array into our list
        
        x_test=vector_list

        predictions = self.model.predict(x_test) #makes the predictions
        
        gold_labels = self.label_encoder.transform(label_strings)
        
        return predictions,gold_labels
    
    def predict_labels_count(self, test_data_path):
        """
        Takes a testfile where tokens and labels are seperated with \t
        returns the sequence of gold-lables and the sequence of predicted labels in INTEGER FORM
        This is the CountVectorizer based implementation
        """
        f = open(test_data_path,'r') #reading in the test data
        input_string = f.read()
        f.close()
        
        input_list=input_string.split("\n") #storing each datum as a string in a list
        input_list=input_list[0:-2] #getting rid of the last empty newline
        feature_strings= []
        label_strings= []
        
        for datum in input_list:
            temp = datum.split("\t") #separating the word from its label
            feature_strings.append(temp[0])
            label_strings.append(temp[1])
        del input_list #deleting the initial list to be economic
        
        x_test = self.vectorizer.transform(feature_strings) #transforming the feature strings into vectors with the count vectorizer


        predictions = self.model.predict(x_test) #makes the predictions
        
        gold_labels = self.label_encoder.transform(label_strings)
        
        return predictions,gold_labels
    


## Training on GloVe embeddings
First we will train our classifier on our training data using the GloVe embeddings. For this we need to read in the embedding dictionary using glove() and then call classifier.train(). This may take a couple of minutes depending on your machine.


In [4]:
classifier = Classifier()
vec= glove('data/dickens_vectors.txt')
classifier.train(vec,'data/train_tokens.txt')


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.1min finished


The classifier has finished training


## Evaluating GloVe embeddings with test data
Now that we have trained the classifier with our custom embeddings, we see how it performs on the test data. The we store that result so we can later compare it to the performance with the CountVectorizer.

In [5]:
predictions = classifier.predict_labels(vec,"data/test_tokens.txt")

glove_f1= f1_score(predictions[1], predictions[0], average='weighted')
glove_accuracy=accuracy_score(predictions[1],predictions[0])

## Training on CountVectorizer embeddings
Now we will start another training sequence, this time based on the CountVectorizer embeddings. This will completely refit the model.

In [6]:
classifier = Classifier()
classifier.train_count('data/count_train_tokens.txt')


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


The classifier has finished training (CountVectorizer)


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.3min finished


## Evaluating the CountVectorizer with the test data
Now we let the newly trained classifier make predictions on the test data again. This time on the basis of the CountVectorizer embeddings.

In [7]:
predictions = classifier.predict_labels_count("data/count_test_tokens.txt")
count_accuracy=accuracy_score(predictions[1],predictions[0])
count_f1=f1_score(predictions[1], predictions[0], average='weighted')

## Comparing the GloVe implementation to the CountVectorizer
Finally, let us compare the two and see which one performs better:

In [8]:
print("Here are the results of the GloVe embeddings:\n","F1_score: ",glove_f1,"\n", "accuracy: ", glove_accuracy,"\n")
print("Here are the results of the CountVectorizer embeddings:\n","F1_score: ",count_f1,"\n", "accuracy: ", count_accuracy)


Here are the results of the GloVe embeddings:
 F1_score:  0.6131187454146257 
 accuracy:  0.5547794117647059 

Here are the results of the CountVectorizer embeddings:
 F1_score:  0.37002266016066365 
 accuracy:  0.3207424098671727


As you can see, the GloVe implementation performs much better. However, even the GloVe implementation does not yield amazing results. This is likely due to the fact that this is quite the difficult task. German, French, and English share many cognates (words that have similar roots). Also, the data contains a variety of names, which are a huge problem. For example "Irene" can be a name in French, English, and German. Considering these problems, the GloVe performance is still pretty good.